In [1]:
!pip install langid
!pip install nltk
!pip install stop_words
!pip install transformers
!pip install torch

In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaConfig, XLMRobertaModel, AutoTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
# Load the dataset into a pandas DataFrame
df = pd.read_excel('com_balanced.xlsx')

In [4]:
df['label'].unique()

array([1, 2, 0], dtype=int64)

In [5]:
new_df = df.rename(columns={"comments": "Phrase", "label": "Sentiment"})

In [8]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', truncation=True, do_lower_case=True)

In [7]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Phrase
        self.targets = self.data.Sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float) }

In [8]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (8394, 2)
TRAIN Dataset: (6715, 2)
TEST Dataset: (1679, 2)


In [9]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [10]:
class XLMRobertaClass(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaClass, self).__init__()
        self.l1 = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [11]:
model = XLMRobertaClass()
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaClass(
  (l1): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [12]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [13]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [14]:
train_accuracy_history = []

# Defining the training function on the 80% of the dataset for tuning the distilbert model
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

        epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    train_accuracy_history.append(epoch_accu)
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [15]:
# CUDA_LAUNCH_BLOCKING=1
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
D:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 1.1482288837432861
Training Accuracy per 5000 steps: 25.0


1679it [2:22:37,  5.10s/it]
0it [00:00, ?it/s]

Training Loss Epoch: 1.074801275352412
Training Accuracy Epoch: 40.65524944154877
Training Loss per 5000 steps: 1.0264960527420044
Training Accuracy per 5000 steps: 25.0


1679it [2:24:32,  5.17s/it]
0it [00:00, ?it/s]

...

Training Loss Epoch: 0.5439648078871029
Training Accuracy Epoch: 79.03201787043932
Training Loss per 5000 steps: 0.2919282913208008
Training Accuracy per 5000 steps: 75.0


1679it [2:23:03,  5.11s/it]
0it [00:00, ?it/s]

Training Loss Epoch: 0.4737173915275457
Training Accuracy Epoch: 81.62323157110946
Training Loss per 5000 steps: 0.33632683753967285
Training Accuracy per 5000 steps: 75.0


1679it [2:22:57,  5.11s/it]

Training Loss Epoch: 0.38359841418537427
Training Accuracy Epoch: 86.10573343261355


In [17]:
import matplotlib.pyplot as plt
# Visualize training history
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracy_history)+1), train_accuracy_history)
plt.xlabel('Training Steps')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy History')

plt.tight_layout()
plt.show()

In [22]:
from sklearn.metrics import accuracy_score, classification_report

def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            # Collect predicted and true labels for accuracy calculation and classification report
            predicted_labels.extend(big_idx.tolist())
            true_labels.extend(targets.tolist())

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")

    # Calculate overall accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    print("Overall Accuracy:", accuracy)

    # Generate classification report
    report = classification_report(true_labels, predicted_labels)
    print("Classification Report:\n", report)

    return accuracy

acc = valid(model, testing_loader)
print("Accuracy on test data = %.2f%%" % (acc * 100))

1it [00:01,  1.61s/it]

Validation Loss per 100 steps: 0.19604216516017914
Validation Accuracy per 100 steps: 0.0


420it [11:23,  1.63s/it]

Overall Accuracy: 0.7772483621203097
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.72      0.76       585
           1       0.74      0.79      0.77       504
           2       0.78      0.82      0.80       590

    accuracy                           0.78      1679
   macro avg       0.78      0.78      0.78      1679
weighted avg       0.78      0.78      0.78      1679

Accuracy on test data = 77.72%


In [30]:
import torch

def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        pad_to_max_length=True,
        return_token_type_ids=True,
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Get the model outputs
    outputs = model(**inputs)

    # Access the model's output based on the model-specific structure
    if isinstance(outputs, tuple):
        model_output = outputs[0]
    else:
        model_output = outputs

    # Calculate the predicted probabilities
    predicted_probabilities = torch.nn.functional.softmax(model_output, dim=1)

    # Get the predicted sentiment label
    predicted_label = torch.argmax(predicted_probabilities, dim=1).item()

    # Map the predicted label to its corresponding word representation
    sentiment_mapping = {0: "neutral", 1: "offense", 2: "sarcasm"}
    predicted_sentiment = sentiment_mapping[predicted_label]

    # Return the text and predicted sentiment label
    return text, predicted_sentiment



## Check comments

In [37]:
text1 = "Мына психологка психолог керек кой"
text2 = "Маған неге қазаққа тұрмыс құрғың келмейді десе  осы интервьюді көрсетем ғой"
text3 = "ДОСқа рахмет соның арқасында Қазакстанда есі дурыс адам барына сенем"
text4 = "Исламда әйел ерінің ата анасына ештеңе міндет емес ұмытпаңдар!"
text5 = "Әйел мындай болу керек, андай болу керек. Қазір әйелді үйде ұстап отыру мүмкін емес."
text6 = "неге мен куйеуге шыккканнан кейин оларды умытып,куйеуим жагына берилип кету керекпин,олар мен ушин не истеди?"
text7 = "Жұлдыздар Дос сияқты қарапайым, шынайы болса екен. Үйренетін дүние көп бұл жігіттен."
text8 = "Анараның ұнайтын жері, қандай қонақ келсе соның вайбына кіріп отырады"



print(predict_sentiment(text1))
print(predict_sentiment(text2))
print(predict_sentiment(text3))
print(predict_sentiment(text4))
print(predict_sentiment(text5))
print(predict_sentiment(text6))
print(predict_sentiment(text7))
print(predict_sentiment(text8))



('Мына психологка психолог керек кой', 'sarcasm')
('Маған неге қазаққа тұрмыс құрғың келмейді десе  осы интервьюді көрсетем ғой', 'neutral')
('ДОСқа рахмет соның арқасында Қазакстанда есі дурыс адам барына сенем', 'neutral')
('Исламда әйел ерінің ата анасына ештеңе міндет емес ұмытпаңдар!', 'neutral')
('Әйел мындай болу керек, андай болу керек. Қазір әйелді үйде ұстап отыру мүмкін емес.', 'neutral')
('неге мен куйеуге шыккканнан кейин оларды умытып,куйеуим жагына берилип кету керекпин,олар мен ушин не истеди?', 'offense')
('Жұлдыздар Дос сияқты қарапайым, шынайы болса екен. Үйренетін дүние көп бұл жігіттен.', 'neutral')
('Анараның ұнайтын жері, қандай қонақ келсе соның вайбына кіріп отырады', 'neutral')


In [1]:
model.save_pretrained('model_directory')

NameError: name 'model' is not defined